In [70]:
import numpy as np
import sklearn as sk
import pandas as pd
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split

In [23]:
x_train_full = pd.read_csv('train.csv')
x_test = pd.read_csv('test.csv')

In [52]:
relevant_features_x = x_train_full[[
    'MSSubClass',
    'MSZoning',
    'LotArea',
    'Utilities',
    'Neighborhood',
    'BldgType',
    'HouseStyle',
    'OverallQual',
    'OverallCond',
    'SalePrice']].copy()

In [110]:
x = relevant_features_x[[
    'LotArea',
    'MSZoning'
]]
y = relevant_features_x.SalePrice

x_train, x_valid, y_train, y_valid = train_test_split(x, y, train_size=0.8, test_size=0.2, random_state=0)

cat_cols = [col for col in x.columns if x[col].dtype == 'object']
num_cols = [col for col in x.columns if x[col].dtype in ['int64', 'float64']]

num_transformer = SimpleImputer(strategy='mean')
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
])
            
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_cols),
    ('cat', cat_transformer, cat_cols)
])

from sklearn.linear_model import Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# model = LinearRegression()
# model = RandomForestRegressor(n_estimators=100, random_state=0)
model = XGBRegressor(n_estimators=1000, learning_rate=0.01)
pipeline = Pipeline(steps=[
    ('preproc', preprocessor),
    ('model', model)
])

In [111]:
pipeline.fit(x_train, y_train)
pred_valid = pipeline.predict(x_valid)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[09:22:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [112]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_valid, pred_valid)
print(mse)

5316030171.931601
